In [96]:
import numpy as np 
from scipy.special import factorial2 as fact2 

#overlap integral between two Gaussian functions of arbitrary momenta

def I(i,j,t,Qx,a,b):
    p = a + b
    q = a*b/p
    if (t < 0) or (t > (i + j)):
        #t for recurrence condition
        #i,j; quantum numbers, Qx; A-B, a,b; exponential 
        return 0.0
    elif i == j == t == 0: 
        return np.exp(-q*Qx*Qx)
    elif j == 0:
        return (1/(2*p))*I(i-1,j,t-1,Qx,a,b) - (q*Qx/a)*I(i-1,j,t,Qx,a,b) + (t+1)*I(i-1,j,t+1,Qx,a,b)
    else:
        return (1/(2*p))*I(i,j-1,t-1,Qx,a,b) + (q*Qx/b)*I(i,j-1,t,Qx,a,b) + (t+1)*I(i,j-1,t+1,Qx,a,b)

def overlap(a,lmn1,A,b,lmn2,B):
    l1,m1,n1 = lmn1
    l2,m2,n2 = lmn2
    S1 = I(l1,l2,0,A[0]-B[0],a,b)
    S2 = I(m1,m2,0,A[1]-B[1],a,b)
    S3 = I(n1,n2,0,A[2]-B[2],a,b)
    return S1*S2*S3*np.power(np.pi/(a+b),1.5)

In [95]:
class BasisFunction:

    def __init__(self,origin,shell,exps,coefs):
        self.origin = np.asarray(origin)
        self.shell = shell
        self.exps = exps
        self.coefs = coefs
        self.norm = None
        self.normalize()

    def normalize(self):
        l,m,n = self.shell
        L = l+m+n
        self.norm = np.sqrt(np.power(2,2*(l+m+n)+1.5)*np.power(self.exps,l+m+n+1.5)\
            /fact2(2*l-1)/fact2(2*m-1)/fact2(2*n-1)/np.power(np.pi,1.5))

        prefactor = np.power(np.pi,1.5)*fact2(2*l-1)/fact2(2*m-1)/fact2(2*n-1)/np.power(2.0,L)

        N = 0.0
        num_exps = len(self.exps)
        for ia in range(num_exps):
            for ib in range(num_exps):
                N += self.norm[ia]*self.norm[ib]*self.coefs[ia]*self.coefs[ib]/np.power(self.exps[ia] + self.exps[ib],L+1.5)

        N *= prefactor
        N = np.power(N,-0.5)
        for ia in range(num_exps):
                self.coefs[ia] *= N

In [131]:
from molecule import Molecule

def read_geom(filename):
    with open(filename, "r") as f:
        content = f.readlines()
    natoms = int(content[0])
    atoms =[]
    for i in range(1,len(content)):
        line = content[i].split()
        if(len(line)>0):
            Z = float(line[0])
            x = 1.88973*float(line[1])
            y = 1.88973*float(line[2])
            z = 1.88973*float(line[3])
        atoms.append([Z,[x,y,z]])
    return atoms

atoms =  read_geom('h2o.xyz')


mol = Molecule(atoms)

print(mol.nre)

<bound method Molecule.nre of <molecule.Molecule object at 0x7fabc1782e80>>


In [132]:
#https://www.basissetexchange.org

H1_o = [0.0, 0.0, 0.88973*0.529]
H2_o = [0.0, 0.0, -0.88973*0.529]


myshell = (0,0,0)

s1exps = [5.447178000, 0.8245472400]
s1coefs = [0.1562849787, 0.9046908767]

s2exps = [0.1831915800]
s2coefs = [1.0]

h1s1 = BasisFunction(H1_o,myshell,s1exps,s1coefs)
h1s2 = BasisFunction(H1_o,myshell,s2exps,s2coefs)

h2s1 = BasisFunction(H2_o,myshell,s1exps,s1coefs)
h2s2 = BasisFunction(H2_o,myshell,s2exps,s2coefs)


basisset = [h1s1,h1s2,h2s1,h2s2]


In [133]:
def S(a,b):
    s = 0.0
    for ia, ca in enumerate(a.coefs):
        for ib,cb in enumerate(b.coefs):
            s += a.norm[ia]*b.norm[ib]*ca*cb*overlap(a.exps[ia],a.shell,a.origin,b.exps[ib],b.shell,b.origin)
    return s

s = S(a,b)
print(s)

0.06361158415932251


In [142]:
def overlap_matrix(basisset):
    nbasis = len(basisset)
    s = np.zeros((nbasis,nbasis))
    for i in range(nbasis):
        for j in range(i+1):
            s[i,j] = s[j,i] = S(basisset[i],basisset[j])
    return s


Overlap = overlap_matrix(basisset)
print(Overlap)


[[1.         0.64589812 0.65347327 0.56487114]
 [0.64589812 1.         0.56487114 0.92204249]
 [0.65347327 0.56487114 1.         0.64589812]
 [0.56487114 0.92204249 0.64589812 1.        ]]


In [144]:
def kinetic(a,lmn1,A,b,lmn2,B):
    l1,m1,n1 = lmn1
    l2,m2,n2 = lmn2
    term0 = b*(2*(l2+m2+n2)+3)*\
                            overlap(a,(l1,m1,n1),A,b,(l2,m2,n2),B)
    term1 = -2*np.power(b,2)*\
                           (overlap(a,(l1,m1,n1),A,b,(l2+2,m2,n2),B) +
                            overlap(a,(l1,m1,n1),A,b,(l2,m2+2,n2),B) +
                            overlap(a,(l1,m1,n1),A,b,(l2,m2,n2+2),B))
    term2 = -0.5*(l2*(l2-1)*overlap(a,(l1,m1,n1),A,b,(l2-2,m2,n2),B) +
                  m2*(m2-1)*overlap(a,(l1,m1,n1),A,b,(l2,m2-2,n2),B) +
                  n2*(n2-1)*overlap(a,(l1,m1,n1),A,b,(l2,m2,n2-2),B))
    return term0+term1+term2

In [143]:
def T(a,b):
    t = 0.0
    for ia, ca in enumerate(a.coefs):
        for ib, cb in enumerate(b.coefs):
            t += a.norm[ia]*b.norm[ib]*ca*cb*\
                     kinetic(a.exps[ia],a.shell,a.origin,\
                     b.exps[ib],b.shell,b.origin)
    return t
t = T(a,b)
print (t)

-0.017560249051917404
